# Product Matching
### Kelompok 1
1. 12S16002 - Diana Pebrianty Pakpahan
2. 12S16022 - Rosa Afresia Siagian
3. 12S16026 - Yolanda Nainggolan
4. 12S16028 - Kaleb Lonari Simanungkalit


### Dataset
Data yang digunakan pada proyek ini berasal dari kaggle (https://www.kaggle.com/PromptCloudHQ/innerwear-data-from-victorias-secret-and-others) yang terdiri dari 9 file csv mengenai produk pakaian dalam.

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re

df_1 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/ae_com.csv')
df_2 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/amazon_com.csv')
df_3 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/btemptd_com.csv')
df_4 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/calvinklein_com.csv')
df_5 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/hankypanky_com.csv')
df_6 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/macys_com.csv')
df_7 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/shop_nordstrom_com.csv')
df_8 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/us_topshop_com.csv')
df_9 = pd.read_csv('C:/Users/Yolanda Nainggolan/Productmatching/victoriassecret_com.csv')

C:\Users\Yolanda Nainggolan\Anaconda3\lib\site-packages\tqdm\std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
dataFrame = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9])
dataFrame.sample(5)

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
417130,Body by Victoria Unlined Demi Bra,$34.50,$34.50,https://www.victoriassecret.com/bras/shop-all-...,Victoria's Secret,Unlined Demi Bra,Victoriassecret US,"A naturally sexy unlined shape, with a lower c...",NaN,NaN,NaN,"[""32A"", ""32B"", ""32C"", ""32D"", ""32DD"", ""32DDD"", ...",36C,Black Mesh
183958,Bombshell NEW! Add-2-Cups Push-Up Bra,$58.50,$58.50,https://www.victoriassecret.com/bras/shop-all-...,Victoria's Secret,Add-2-Cups Push-Up Bra,Victoriassecret US,The ultimate lift-loving push-up is better tha...,NaN,NaN,NaN,"[""30AA"", ""30A"", ""30B"", ""30C"", ""30D"", ""30DD"", ""...",38DD,Coconut White Wildflower Lace Back
30928,The T-Shirt Demi Bra,$42.50,$42.50,https://www.victoriassecret.com/bras/shop-all-...,Victoria's Secret,Demi Bra,Victoriassecret US,"As comfy as your favorite tee, this demi is an...",4.4,149.0,NaN,"[""32A"", ""32B"", ""32C"", ""32D"", ""32DD"", ""32DDD"", ...",36D3,Black Matte Solid
408076,PINK Lace T-Shirt Bra,$32.50,$20.00,https://www.victoriassecret.com/pink/all-bras/...,Victoria's Secret Pink,Lace T-Shirt Bra,Victoriassecret US,An everyday bra that essential for every bra w...,NaN,NaN,NaN,"[""30AA"", ""30A"", ""30B"", ""30C"", ""30D"", ""30DD"", ""...",36A,amethyst print
5576,Stretch Cotton Bralette,$68.00,$68.00,http://shop.nordstrom.com/s/hanky-panky-stretc...,HANKY PANKY,Bras,Nordstrom US,Comfy and cute for lounging or layering in sup...,NaN,NaN,"[""Slips on over head"", ""Removable soft cups"", ...","[""Medium"", ""Small"", ""X-Small""]","[""Medium"", ""Small"", ""X-Small""]",Chai


## 2. Data Preprocessing

In [3]:
dataFrame = dataFrame[['product_name', 'description']]
dataFrame = dataFrame[pd.notnull(dataFrame['description'])]
dataFrame.sample(5)

,product_name,description
365298,PINK NEW! Ultimate Push-Up Strappy Sports Bra,Work it! This longline bralette features a str...
167801,Victoria Sport Incredible by Victoria Sport Bra,Tackle high-intensity workouts in a truly Incr...
363032,Very Sexy Satin & Lace Babydoll,"This satin babydoll is anything but shy, with ..."
331794,The Bralette Collection NEW! Scalloped Triangl...,"Sleek and smooth: this raw-cut bralette, in a ..."
10344,Calvin Klein Women's Seductive Comfort Lift St...,This strapless multiway bra features seamless ...


In [4]:
dataFrame.shape

(613143, 2)

In [5]:
dataFrame.isnull().sum()

product_name    0
description     0
dtype: int64

#### Catatan:
Karena jumlah data sangat besar, kita hanya memakai 10.000 data yang akan dipilih secara random

In [6]:
df = dataFrame.sample(n=10000, random_state=42)
df.sample(10)

,product_name,description
296409,Victoria Sport NEW! Incredible by Victoria Spo...,Tackle high-intensity workouts in a truly Incr...
16884,Perfectly Fit Memory Touch Push-Up Bra QF1120,"Lightweight, natural support gets the classic,..."
2632,b.tempt'd Lace Kiss Thong,This is a wearable way to WOW in any situation...
366785,Victoria Sport Lightweight by Victoria Sport Bra,Our #1 sport bra for a reason: with a supersof...
396428,The Bralette Collection Lace Racerback Bralette,Upgrade your everyday with pretty scalloped la...
6937,Aerie Boho Racerback Bralette,"Peace, love and lace. Wear often. Feel good! s..."
330567,Sexy Tee Demi Bra,"Sleek and lightweight, this low-cut demi is de..."
7834,Calvin Klein Women's Perfectly Fit Lightly Lin...,This full coverage bra features lightly lined ...
36224,Logo Bikini F3787,You'll want this classic style in your collect...
27532,Calvin Klein Women's 5 Pack Cotton Stretch Log...,"This essential, bikini 5-pack features 5 pairs..."


In [7]:
df.shape

(10000, 2)

In [8]:
df.index = range(10000)
df['description'].apply(lambda x: len(x.split(' '))).sum()

564187

In [9]:
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text)
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['description'] = df['description'].apply(cleanText)

In [10]:
def print_description(index):
    example = df[df.index == index][['description', 'product_name']].values[0]
    if len(example) > 0:
        print('Description:\n', example[0])
        print('\n product_name:', example[1])
        
print_description(12)

Description:
 our #1 push-up is seier than ever with plush, even softer padding for the lift and cleavage you love and a more natural feel, plus edges that disappear under clothes.lift & lining push-up padding adds 1 cup size underwire cupsstraps & hooks fully adjustable straps back closure double row of hook-and-eye closures 4 settings to ensure a perfect fit can be worn classic or crossbackdetails & fabric supersmooth neckline disappears under clothes cutouts between cups vs charm on side of cup imported nylon/spande 

 product_name: Very Sexy Push-Up Bra


In [11]:
print_description(20)

Description:
 pretty lace makes the silky-soft cotton panty you love all the more feminine, in colors and prints to brighten your lingerie drawer. lace trim low rise moderate back coverage: shows a little, but not all machine wash imported cotton/nylon/spande

 product_name: Cotton Lingerie NEW! Lace-waist String Bikini Panty


## 3. Features Extraction

In [12]:
train, test = train_test_split(df, test_size=0.3, random_state=42)
import nltk
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['description']), tags=[r.product_name]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['description']), tags=[r.product_name]), axis=1)

In [13]:
train_tagged.values[30]

TaggedDocument(words=['an', 'everyday', 'bra', 'that', 'essential', 'for', 'every', 'bra', 'wardrobe', 'convertible', 'straps', 'make', 'it', 'perfect', 'for', 'wearing', 'with', 'any', 'tank', 'or', 't-shirt', 'lightly', 'lined', 'for', 'subtle', 'lift', 'underwire', 'cups', 'light', 'push', 'fully', 'adjustable', 'straps', 'can', 'convert', 'to', 'crossback', 'and', 'snap', 'into', 'place', 'for', 'secure', 'hold', 'back', 'closure', 'hook', 'and', 'eye', 'closure', 'imported', 'polyamide/spande'], tags=['PINK NEW! Wear Everywhere T-Shirt Bra'])

In [14]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [15]:
model = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:00<?, ?it/s]


In [16]:
len(model.docvecs)

1297

In [17]:
model.wv.vocab.keys()

dict_keys(['our', 'sport', 'bra', 'for', 'reason', 'with', 'supersoft', 'elastic', 'band', 'and', 'wireless', 'cups', 'that', 'breathe', 'easy', 'this', 'medium-support', 'favorite', 'is', 'ready', 'to', 'bike', 'bo', 'train', 'hard', 'details', 'medium', 'support', 'unstructured', 'design', 'adjustable', 'straps', 'no', 'clasps', 'or', 'hooks', 'strappy', 'back', 'performance', 'fabric', 'perfect', 'spinning', 'kickboing', 'circuit', 'training', 'super-soft', 'comfort', 'movement', 'lightweight', 'breathable', 'body-wick', 'keeps', 'you', 'cool', 'dry', 'machine', 'wash.', 'tumble', 'imported', 'polyamide/spande', 'best', 'selling', 'convertible', 'push-up', 'available', 'in', 'basic', 'colors', 'new', 'fashion', 'color', 'no-show', 'fit—even', 'under', 'your', 'tightest', 'clothes', 'barely-there', 'thong', 'features', 'sporty', 'logo', 'waist', 'soft', 'cotton', 'only', 'by', 'victoria', 'secret', 'pink', 'iconic', 'waistband', 'etra', 'low', 'rise', 'cotton/elastane/polyamide/polye

In [18]:
model.most_similar('fabric')

C:\Users\Yolanda Nainggolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('repay', 0.18559274077415466),
 ('midriff', 0.1787772923707962),
 ('flirty', 0.16629986464977264),
 ('synthetic', 0.16413606703281403),
 ('sweaters', 0.15486739575862885),
 ('so', 0.15115562081336975),
 ('simple', 0.1508401483297348),
 ('few', 0.1496153026819229),
 ('pearls', 0.14841711521148682),
 ('1½', 0.14837251603603363)]

In [19]:
%%time
for epoch in range(30):
    model.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

100%|██████████████████████████████████████████████████████████████████████████| 7000/7000 [00:00<00:00, 923274.47it/s]


Wall time: 26.6 s


In [ ]:
model.save("model.doc2vec")

In [ ]:
model = gensim.models.Doc2Vec.load('model.doc2vec')  

In [23]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train, X_train = vec_for_learning(model, train_tagged)
y_test, X_test = vec_for_learning(model, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

C:\Users\Yolanda Nainggolan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Yolanda Nainggolan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## 4. Kalkulasi Kemiripan

In [20]:
def matching(inputs):
    new_sentence = inputs.split(" ")
    print("Berikut adalah 10 produk yang mirip")
    tag = model.docvecs.most_similar(positive=[model.infer_vector(new_sentence)],topn=10)
    
    return tag

In [21]:
matching("Dream Angels Lace Bustier")

Berikut adalah 10 produk yang mirip


[('Body by Victoria Cheekini Panty', 0.0846535861492157),
 ('Very Sexy Strappy Lace Cheeky Panty', 0.0774226114153862),
 ('Body by Victoria Thong Panty', 0.07572682201862335),
 ('Cotton Lingerie V-string Panty', 0.07400381565093994),
 ('PINK NEW! Super Soft Lace Trim Thong', 0.07398561388254166),
 ('Very Sexy Strappy Lace Thong Panty', 0.07314929366111755),
 ('PINK NEW! Wear Everywhere Logo Trim Hipster', 0.06827020645141602),
 ('PINK Super Soft Lace Trim Thong', 0.06763197481632233),
 ('Cotton Lingerie NEW! Lace-waist Thong Panty', 0.06280837953090668),
 ('Very Sexy NEW! Strappy Lace Thong Panty', 0.06242937222123146)]

In [22]:
matching("clothes Cutouts between cups VS charm on side of cup")

Berikut adalah 10 produk yang mirip


[('PINK NEW! Lace Back No-Show Cheekster', 0.9154719114303589),
 ('PINK NEW! Ultimate Lace-Up Plunge Sports Bra', 0.9068278074264526),
 ('PINK Ultimate Lace-Up Plunge Sports Bra', 0.9013830423355103),
 ('Aerie Play Mesh Back Sports Bra', 0.8929629325866699),
 ('Signature Lace Melanie Bra', 0.8910152316093445),
 ('PINK Eyelash Lace Cheekster', 0.8823460340499878),
 ('Chalk Stripe Cheeky Hipster', 0.8739190101623535),
 ('Very Sexy NEW! Satin Midi Slip', 0.8712292909622192),
 ('Hannah Cotton Lightly Lined Bra', 0.8634597063064575),
 ('Very Sexy Crossback Satin Slip', 0.862072229385376)]